In [15]:
import torch

print("Is CUDA available?", torch.cuda.is_available())
print("CUDA device count:", torch.cuda.device_count())
print("Current CUDA device:", torch.cuda.current_device())
print("CUDA device name:", torch.cuda.get_device_name(torch.cuda.current_device()))


Is CUDA available? True
CUDA device count: 1
Current CUDA device: 0
CUDA device name: NVIDIA GeForce GTX 1650


Parse the JSON Annotations

In [9]:
import os
import json
from pathlib import Path
import shutil

# Paths
train_ann_dir = 'mvtec-ad-DatasetNinja/train/ann'
train_img_dir = 'mvtec-ad-DatasetNinja/train/img'
test_ann_dir = 'mvtec-ad-DatasetNinja/test/ann'
test_img_dir = 'mvtec-ad-DatasetNinja/test/img'

# Output directories for classified images
data_dir = 'classified_torch_data'
Path(data_dir).mkdir(parents=True, exist_ok=True)
for split in ['train', 'test']:
    for label in ['defective', 'non-defective']:
        Path(os.path.join(data_dir, split, label)).mkdir(parents=True, exist_ok=True)

# Function to parse annotations and move images to classified folders
def process_annotations(ann_dir, img_dir, split):
    for ann_file in os.listdir(ann_dir):
        if ann_file.endswith('.json'):
            with open(os.path.join(ann_dir, ann_file), 'r') as f:
                annotation = json.load(f)
            
            # Extract the base filename without the .json extension
            img_filename = ann_file.replace('.json', '')
            
            # Check for image with the exact filename from the JSON
            img_path = os.path.join(img_dir, img_filename)
            if not os.path.exists(img_path):
                # Try adding common extensions if the exact filename doesn’t work
                img_path = os.path.join(img_dir, f"{img_filename}.jpg")
                if not os.path.exists(img_path):
                    img_path = os.path.join(img_dir, f"{img_filename}.png")
            
            # Check if the image file exists
            if os.path.exists(img_path):
                # Check for defects in annotation
                is_defective = any(obj['classTitle'] != 'good' for obj in annotation.get('objects', []))
                label = 'defective' if is_defective else 'non-defective'
                
                # Destination path
                dest_path = os.path.join(data_dir, split, label, os.path.basename(img_path))
                
                # Copy the image to the respective folder
                shutil.copy(img_path, dest_path)
                print(f"Copied {img_filename} to {dest_path} as {label}")
            else:
                print(f"Image file for {img_filename} not found. Skipping...")

# Process both train and test annotations
process_annotations(train_ann_dir, train_img_dir, 'train')
process_annotations(test_ann_dir, test_img_dir, 'test')


Copied bottle_good_000.png to classified_torch_data\train\non-defective\bottle_good_000.png as non-defective
Copied bottle_good_001.png to classified_torch_data\train\non-defective\bottle_good_001.png as non-defective
Copied bottle_good_002.png to classified_torch_data\train\non-defective\bottle_good_002.png as non-defective
Copied bottle_good_003.png to classified_torch_data\train\non-defective\bottle_good_003.png as non-defective
Copied bottle_good_004.png to classified_torch_data\train\non-defective\bottle_good_004.png as non-defective
Copied bottle_good_005.png to classified_torch_data\train\non-defective\bottle_good_005.png as non-defective
Copied bottle_good_006.png to classified_torch_data\train\non-defective\bottle_good_006.png as non-defective
Copied bottle_good_007.png to classified_torch_data\train\non-defective\bottle_good_007.png as non-defective
Copied bottle_good_008.png to classified_torch_data\train\non-defective\bottle_good_008.png as non-defective
Copied bottle_good_

In [10]:
import os
import random
import shutil

# Define paths
test_defective_dir = 'classified_torch_data/test/defective'
train_defective_dir = 'classified_torch_data/train/defective'

# Ensure the train_defective directory exists
os.makedirs(train_defective_dir, exist_ok=True)

# List all files in test_defective directory
test_defective_files = os.listdir(test_defective_dir)

# Specify the portion to move (e.g., 80% of the test defective images)
portion_to_move = 0.9
num_files_to_move = int(len(test_defective_files) * portion_to_move)

# Select random files to move
files_to_move = random.sample(test_defective_files, num_files_to_move)

# Move files from test/defective to train/defective
for filename in files_to_move:
    src_path = os.path.join(test_defective_dir, filename)
    dest_path = os.path.join(train_defective_dir, filename)
    shutil.move(src_path, dest_path)
    print(f"Moved {filename} from test/defective to train/defective")

print(f"Moved {num_files_to_move} files to train/defective.")


Moved carpet_thread_015.png from test/defective to train/defective
Moved toothbrush_defective_025.png from test/defective to train/defective
Moved hazelnut_hole_001.png from test/defective to train/defective
Moved bottle_broken_large_015.png from test/defective to train/defective
Moved hazelnut_print_007.png from test/defective to train/defective
Moved hazelnut_print_006.png from test/defective to train/defective
Moved leather_glue_009.png from test/defective to train/defective
Moved cable_cable_swap_005.png from test/defective to train/defective
Moved transistor_cut_lead_001.png from test/defective to train/defective
Moved leather_glue_001.png from test/defective to train/defective
Moved wood_combined_000.png from test/defective to train/defective
Moved grid_bent_007.png from test/defective to train/defective
Moved tile_oil_012.png from test/defective to train/defective
Moved tile_oil_014.png from test/defective to train/defective
Moved grid_broken_003.png from test/defective to train

Load and Preprocess the Images

In [39]:
import os

# Paths
train_defective_dir = 'classified_torch_data/train/defective'
train_non_defective_dir = 'classified_torch_data/train/non-defective'
test_defective_dir = 'classified_torch_data/test/defective'
test_non_defective_dir = 'classified_torch_data/test/non-defective'

# Count images in each directory
num_train_defective = len(os.listdir(train_defective_dir))
num_train_non_defective = len(os.listdir(train_non_defective_dir))
num_test_defective = len(os.listdir(test_defective_dir))
num_test_non_defective = len(os.listdir(test_non_defective_dir))

# Print the counts
print("Training Set:")
print(f"  Defective: {num_train_defective}")
print(f"  Non-defective: {num_train_non_defective}")

print("\nTesting Set:")
print(f"  Defective: {num_test_defective}")
print(f"  Non-defective: {num_test_non_defective}")

# Total
total_images = num_train_defective + num_train_non_defective + num_test_defective + num_test_non_defective
print("\nTotal images in dataset:", total_images)


Training Set:
  Defective: 1132
  Non-defective: 3629

Testing Set:
  Defective: 126
  Non-defective: 467

Total images in dataset: 5354


Better Model

In [4]:
# Import necessary libraries
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from sklearn.metrics import classification_report, confusion_matrix
from tqdm import tqdm  # For progress bars
from torchvision.models import ResNet18_Weights

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Data Paths
train_dir = 'classified_torch_data/train'
test_dir = 'classified_torch_data/test'

# 1. Define Data Transformations
# Dynamic augmentations for training
transform_train = transforms.Compose([
    transforms.RandomRotation(30),
    transforms.RandomHorizontalFlip(),
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Simple transformations for testing
transform_test = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# 2. Load the Dataset
train_data = datasets.ImageFolder(root=train_dir, transform=transform_train)
test_data = datasets.ImageFolder(root=test_dir, transform=transform_test)

# 3. Data Loaders with Efficient Settings
train_loader = DataLoader(train_data, batch_size=32, shuffle=True, num_workers=2, pin_memory=True)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False, num_workers=2, pin_memory=True)

# 4. Initialize ResNet18 Model with Fine-Tuning
model = models.resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)

# Modify the final layer for binary classification
model.fc = nn.Linear(model.fc.in_features, 2)
model = model.to(device)

# 5. Set up a Weighted Loss Function
# Calculate class weights inversely proportional to class frequencies
defective_count = len(os.listdir(os.path.join(train_dir, "defective")))
non_defective_count = len(os.listdir(os.path.join(train_dir, "non-defective")))
class_weights = torch.tensor([
    1.0 / defective_count, 
    1.0 / non_defective_count
], dtype=torch.float32).to(device)

# Use weighted cross-entropy loss
criterion = nn.CrossEntropyLoss(weight=class_weights)

# 6. Set Up Optimizer with Fine-Tuning
# Freeze earlier layers and fine-tune the last block
for param in model.parameters():
    param.requires_grad = False
for param in model.layer4.parameters():
    param.requires_grad = True

optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-4)

# 7. Training Loop with Progress Bar
epochs = 20
for epoch in range(epochs):
    model.train()
    running_loss = 0.0

    with tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}", unit="batch") as pbar:
        for images, labels in pbar:
            images, labels = images.to(device), labels.to(device)

            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)

            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            pbar.set_postfix(loss=running_loss / (pbar.n + 1))

    print(f"Epoch [{epoch+1}/{epochs}], Loss: {running_loss/len(train_loader):.4f}")

# 8. Evaluation Loop with Progress Bar
model.eval()
all_labels = []
all_preds = []

with torch.no_grad():
    with tqdm(test_loader, desc="Evaluating", unit="batch") as pbar:
        for images, labels in pbar:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)

            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(predicted.cpu().numpy())

# 9. Classification Report and Confusion Matrix
print("\nClassification Report:")
print(classification_report(all_labels, all_preds, target_names=train_data.classes))

cm = confusion_matrix(all_labels, all_preds)
print("\nConfusion Matrix:")
print(cm)

# 10. Save the Trained Model
torch.save(model.state_dict(), 'industrial_defect_classifier.pth')
print("Model saved as 'industrial_defect_classifier.pth'")


Using device: cuda


Epoch 1/20: 100%|██████████| 149/149 [01:28<00:00,  1.68batch/s, loss=0.577]


Epoch [1/20], Loss: 0.5773


Epoch 2/20: 100%|██████████| 149/149 [01:28<00:00,  1.68batch/s, loss=0.44] 


Epoch [2/20], Loss: 0.4371


...

Epoch [19/20], Loss: 0.1678


Epoch 20/20: 100%|██████████| 149/149 [01:50<00:00,  1.35batch/s, loss=0.174]


Epoch [20/20], Loss: 0.1726


Evaluating: 100%|██████████| 19/19 [00:16<00:00,  1.15batch/s]


Classification Report:
               precision    recall  f1-score   support

    defective       0.94      0.82      0.88       126
non-defective       0.95      0.99      0.97       467

     accuracy                           0.95       593
    macro avg       0.95      0.90      0.92       593
 weighted avg       0.95      0.95      0.95       593


Confusion Matrix:
[[103  23]
 [  6 461]]
Model saved as 'industrial_defect_classifier.pth'
